In [1]:
import zmq
import time
import socket
import os
import sys
import logging
import datetime
import threading
import warnings
import schedule
import numpy as np
from numpy import matlib

from multiprocessing import Process

# setup the logger
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

# Check wich computer to decide where the things are mounted
comp_name=socket.gethostname()
logger.info('Computer: ' + comp_name)

from rigmq.host import sbc

2019-06-14 14:39:01,989 root         INFO     Computer: zufurious


### Session tools and methods

In [2]:
# open the devices
pi_zmq = sbc.SBC(port='5558', ip='192.168.1.50') # raspi on main lab
pi_zmq.connect()

In [3]:
# make a rig dictionary
# soon to be object
rig_par = {'sbc': pi_zmq,
          'oe': None}

data_path = os.path.join('E:\\' , 'newData', 'song_recordings')
bird = 'b11k10'

## very simple single trial tests

In [4]:
def cmd_trial_number(tr_num):
    #on is boolean
    return 'trial_number number {}'.format(int(tr_num)).encode('utf8')
        
def cmd_switch_glass(switch: str) -> str:
    return 'glass switch {}'.format(switch).encode('utf8')

def run_trial(rig_par={'sbc': None, 'oe':None}, 
              trial_par={'iti':120*1000., 'tr_num': 5, 'wave_file': None, 'transparent_time': 10*1000}):
    time.sleep(int(trial_par['iti']*100)/100000.)
    # a very simple trial; switch on, wait, switch off
    rig_par['sbc'].send_command(cmd_trial_number(trial_par['tr_num']))
    pi_zmq.send_command(cmd_switch_glass('on'))
    time.sleep(int(trial_par['transparent_time']*100)/100000.)
    pi_zmq.send_command(cmd_switch_glass('off'))
    
class Block:
    # A block is a generator that gives trials (dictionaries)
    def __init__(self, transp_time_list, iti_bounds_ms, size, random = True):
        self.trials = 0
        self.iti_bounds = iti_bounds_ms
        self.stim_list = transp_time_list
        self.size = size
        self.trial_pars = {'iti': None, 'tr_num': 0, 'stim': None, 'transparent_time': 10*1000}
        self.stim_order = matlib.repmat(np.arange(len(transp_time_list)),
                                           int(np.ceil(size/len(transp_time_list))),
                                           1)[:size].flatten()
        if random:
            np.random.shuffle(self.stim_order)
    
    def __iter__(self,):
        return self
    
    def next(self, ):
        if self.trials < self.size:
            self.trial_pars['iti'] = np.random.randint(self.iti_bounds[0], self.iti_bounds[1])
            #print self.trials
            self.trial_pars['tr_num'] = self.trials
            self.trial_pars['stim'] = None
            self.trial_pars['transparent_time'] = self.stim_list[self.stim_order[self.trials]]
            self.trials+=1
            return self.trial_pars
        else:
            raise StopIteration()

### Create a block of trials that switch on the glass for some time

In [5]:
block_times = [10*1000, 20*1000, 30*1000, 60*1000] # open times are 10, 20, 30, 60 seconds
block_iti_bounds = [5*60*1000, 15*60*1000] # it ranges from 5 to 15 min, uniformly distributed
a_block = Block(block_times, block_iti_bounds, 80) # block is 80 trials, that about 14 hour.

#### spit one trial dict

In [6]:
a_block.next()

{'iti': 501275, 'tr_num': 0, 'stim': None, 'transparent_time': 30000}

### Make a Runner Thread that goes through a block running each trial

In [7]:
class Runner(threading.Thread):
    # runs an experiment block
    # forced to no recording system to control
    def __init__(self, block, rig_par, recorder=None, group=None, target=None, name=None, 
                 args=(), kwargs=None):
        threading.Thread.__init__(self, group=group, target=target, name=name)
        
        logging.info('Initializing block size {}'.format(block.size))
        self.block = block #block object
        self.rig = rig_par
        self.stim_sys = rig_par['sbc']
        self.rec_sys = None
        self.running = threading.Event() #Event
        self.finished = False
        self.trial = None
        self.running.clear()
        self.recorder = recorder #Record control
   
    def run(self):
        logging.debug('running block')
        if not self.running.is_set():
            logger.info('Starting Block')
#             if self.is_recording()==True:
#                 logging.debug('Starting recording')
#                 self.recorder.start()
#                 time.sleep(2)
            self.running.set()
            while True:
                try:
                    if self.running.is_set():
                        self.trial = self.block.next()
                        logging.info('Running trial {}'.format(self.trial))
                        #print(self.trial)
                        run_trial(rig_par=self.rig, trial_par=self.trial)
                    else:
                        self.end_block()
                        break

                except StopIteration:
                    self.finished = True
                    self.running.clear()
                    logger.info('finished block')
                    break
            
            if self.is_recording()==True:
                logger.info('Runner stopping recording')
                self.recorder.signal_stop()
        else:
            logging.info('Cant start, already running')
            
        return self.finished
                    
    def end_block(self):
        print('stopping the block in trial {}'.format(self.trial['tr_num']))
        self.finished = False
        
    def signal_stop(self):
        self.running.clear()
        
    def is_running(self):
        return self.running.is_set()
    
    def is_recording(self):
        logging.debug('runner checking recording status')
        logging.debug('{}'.format(self.recorder))
        if self.recorder is not None:
            logging.debug('status {}'.format(self.recorder.report_recording()))
            return self.recorder.report_recording()
        else:
            return None

#### Run a block till the end

In [8]:
screen_block = Block(block_times, block_iti_bounds, 80)
block_runner = Runner(screen_block, rig_par, recorder=None)
block_runner.start()

2019-06-14 11:57:35,832 root         INFO     Initializing block
2019-06-14 11:57:35,833 root         INFO     Starting Block
2019-06-14 11:57:35,839 root         INFO     Running trial {'iti': 848808, 'tr_num': 0, 'stim': None, 'transparent_time': 10000}


{'iti': 848808, 'tr_num': 0, 'stim': None, 'transparent_time': 10000}


## schedule a block to start every day at 6am
to do a simple, dayly scehduled ~14hr block, I use schedule package
https://schedule.readthedocs.io/en/stable/

In [20]:
def do_block():
    logger = logging.getLogger() 
    logger.info('Starting a new block')
    bird_pybeh_path = os.path.join(data_path, bird, 'PyBehave', datetime.datetime.today().strftime('%Y-%m-%d'))
    os.makedirs(bird_pybeh_path, exist_ok=True)
    block_log_path = os.path.join(bird_pybeh_path, 'smartglass_log.txt')
    logger.info('Logging activity to {}'.format(block_log_path))
    
    # add the file log handler
    f_handler = logging.FileHandler(block_log_path)
    f_handler.setFormatter(formatter)
    logger.addHandler(f_handler)
    logger.removeHandler(handler)
    
    # run the block, join the thread.
    screen_block = Block(block_times, block_iti_bounds, 80)
    logger.info(' here starts block')
    block_runner = Runner(screen_block, rig_par, recorder=None) #remember runner is a Thread subclass
    block_runner.start()
    block_runner.join()
    
    # remove the file handler and reactivate the stream_handler
    logger.addHandler(handler)
    logger.removeHandler(f_handler)
    logger.info('this should print on scren')


do_block()

2019-06-16 11:04:24,760 root         INFO     Starting a new block
2019-06-16 11:04:24,772 root         INFO     Logging activity to E:\newData\song_recordings\b11k10\PyBehave\2019-06-16\smartglass_log.txt
2019-06-16 20:51:10,222 root         INFO     this should print on scren
2019-06-17 06:00:00,386 schedule     INFO     Running job Every 1 day at 06:00:00 do do_block() (last run: [never], next run: 2019-06-17 06:00:00)
2019-06-17 06:00:00,386 root         INFO     Starting a new block
2019-06-17 06:00:00,387 root         INFO     Logging activity to E:\newData\song_recordings\b11k10\PyBehave\2019-06-17\smartglass_log.txt
2019-06-17 16:16:50,446 root         INFO     this should print on scren


In [10]:
schedule.jobs

[Every 1 day at 06:00:00 do do_block() (last run: [never], next run: 2019-06-15 06:00:00)]

### Leave the scheduler running

In [19]:
def run_schedule():
    logger.info('Will start the schedule on a Process')
    logger.info('Sheduled tasks list: {}'.format(schedule.jobs))
    while True:
        schedule.run_pending()
        time.sleep(1)

## leave it running on is onwn thread
schedule.clear()
schedule.every().day.at("06:00").do(do_block)
schedule_thread = threading.Thread(target=run_schedule)
schedule_thread.start()

2019-06-16 11:04:14,448 root         INFO     Will start the schedule on a Process
2019-06-16 11:04:14,453 root         INFO     Sheduled tasks list: [Every 1 day at 06:00:00 do do_block() (last run: [never], next run: 2019-06-17 06:00:00)]


In [14]:
schedule_process.pid

58124

In [17]:
schedule.jobs

[Every 1 day at 06:00:00 do do_block() (last run: [never], next run: 2019-06-15 06:00:00)]

### Debugging lines

In [27]:
pi_zmq.send_command(cmd_switch_glass('on'))

'smartglass_switched on'

In [28]:
pi_zmq.send_command(cmd_switch_glass('off'))

'smartglass_switched off'

In [18]:
pi_zmq.send_command(cmd_trial_number(5))

'ok trial_number:5'

In [19]:
os.makedirs( 'tuputamadre'))

In [20]:
os.path.join('E', 'newData', 'tuputamadre')

'E\\newData\\tuputamadre'